In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import cvxpy as cp
import numpy as np
from gcspy import GraphOfConvexSets

In [ ]:
visit_rooms = [0, 2, 7, 10, 13, 15, 18]
rooms = [
    (0, [0, 5], [2, 8]),
    (1, [0, 3], [4, 5]),
    (2, [0, 0], [2, 3]),
    (3, [2, 5], [4, 8]),
    (4, [2, 0], [4, 3]),
    (5, [4, 0], [6, 8]),
    (6, [6, 7], [8, 8]),
    (7, [6, 1], [8, 7]),
    (8, [6, 0], [8, 1]),
    (9, [8, 0], [10, 10]),
    (10, [10, 5], [12, 10]),
    (11, [10, 4], [12, 5]),
    (12, [10, 0], [12, 4]),
    (13, [12, 0], [16, 2]),
    (14, [12, 2], [14, 10]),
    (15, [14, 7], [16, 10]),
    (17, [0, 8], [4, 10]),
    (18, [4, 8], [8, 10]),
    (19, [14, 2], [16, 7]),
#     (20, [6, 8], [8, 9]),
]

doors = [
    (0, 3, [2, 7], [2, 8]),
    (1, 2, [1, 3], [2, 3]),
    (1, 5, [4, 4], [4, 5]),
    (2, 4, [2, 2], [2, 3]),
    (3, 5, [4, 5], [4, 8]),
    (4, 5, [4, 0], [4, 3]),
    (5, 6, [6, 7], [6, 8]),
    (5, 7, [6, 1], [6, 2]),
    (5, 8, [6, 0], [6, 1]),
#     (5, 18, [5, 9], [6, 9]),
    (6, 9, [8, 7], [8, 8]),
    (7, 9, [8, 3], [8, 4]),
    (8, 9, [8, 0], [8, 1]),
    (9, 10, [10, 7], [10, 8]),
    (9, 11, [10, 4], [10, 5]),
    (9, 12, [10, 3], [10, 4]),
    (9, 18, [8, 9], [8, 10]),
    (10, 14, [12, 5], [12, 6]),
    (11, 14, [12, 4], [12, 5]),
    (12, 13, [12, 0], [12, 2]),
    (14, 15, [14, 7], [14, 8]),
    (14, 19, [14, 6], [14, 7]),
    (3, 17, [3, 8], [4, 8]),
#     (18, 20, [6, 9], [8, 9]),
]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_room(n, l, u):
    l = np.array(l)
    u = np.array(u)
    d = u - l
    fc = "mistyrose" if n in visit_rooms else "mintcream"
    rect = patches.Rectangle(l, *d, fc=fc, ec="k")
#     c = (l + u) / 2
#     plt.text(*c, n, zorder=3, ha='center', va='center')
    plt.gca().add_patch(rect)
    
def plot_door(l, u):
    endpoints =  np.array([l, u]).T
    plt.plot(*endpoints, color="mintcream", solid_capstyle="butt")
    plt.plot(*endpoints, color="grey", linestyle=":")
    
def plot_floor(rooms, doors):
    plt.gca().set_aspect('equal')
    plt.axis("off")
    for room in rooms:
        plot_room(*room)
    for door in doors:
        plot_door(door[2], door[3])
        
plt.figure()
plot_floor(rooms, doors)

In [ ]:
class Floor(GraphOfConvexSets):

    def __init__(self, rooms, doors, name):
        super().__init__()
        self.name = name
        for room in rooms:
            self.add_room(*room)
        for door in doors:
            self.add_door(*door)
        
    def add_room(self, n, l, u):
        v = self.add_vertex(f"{self.name}_{n}")
        x1 = v.add_variable(2)
        x2 = v.add_variable(2)
        v.add_constraint(x1 >= l)
        v.add_constraint(x2 >= l)
        v.add_constraint(x1 <= u)
        v.add_constraint(x2 <= u)
        v.add_cost(cp.norm(x2 - x1, 2))
        return v
    
    def add_door(self, n, m, l, u):
        e1 = self.add_one_way_door(n, m, l, u)
        e2 = self.add_one_way_door(m, n, l, u)
        return e1, e2
    
    def add_one_way_door(self, n, m, l, u):
        tail = self.get_vertex_by_name(f"{self.name}_{n}")
        head = self.get_vertex_by_name(f"{self.name}_{m}")
        e = self.add_edge(tail, head)
        e.add_constraint(tail.variables[1] == head.variables[0])
        e.add_constraint(tail.variables[1] >= l)
        e.add_constraint(tail.variables[1] <= u)
        return e

In [ ]:
gcs = GraphOfConvexSets()

num_floors = len(visit_rooms)
for floor in range(num_floors):
    gcs.add_subgraph(Floor(rooms, doors, floor))

def connect_floors(floor1, floor2, room):
    tail = gcs.get_vertex_by_name(f"{floor1}_{room}")
    head = gcs.get_vertex_by_name(f"{floor2}_{room}")
    edge = gcs.add_edge(tail, head)
    edge.add_constraint(tail.variables[1] == head.variables[0])

first_room = visit_rooms[0]
first_floor = 0
last_floor = num_floors - 1
connect_floors(last_floor, first_floor, first_room)
for floor in range(last_floor):
    for room in visit_rooms[1:]:
        connect_floors(floor, floor + 1, room)

In [ ]:
ilp_constraints = []
yv = gcs.vertex_binaries()
ye = gcs.edge_binaries()

for i, v in enumerate(gcs.vertices):
    inc_edges = gcs.incoming_indices(v)
    out_edges = gcs.outgoing_indices(v)
    ilp_constraints.append(yv[i] == sum(ye[inc_edges]))
    ilp_constraints.append(yv[i] == sum(ye[out_edges]))
    
def get_flow(floor1, floor2, room):
    tail_name = f"{floor1}_{room}"
    head_name = f"{floor2}_{room}"
    edge = gcs.get_edge_by_name(tail_name, head_name)
    return ye[gcs.edge_index(edge)]

ilp_constraints.append(get_flow(last_floor, first_floor, first_room) == 1)
for room in visit_rooms[1:]:
    flow = sum(get_flow(floor, floor + 1, room) for floor in range(last_floor))
    ilp_constraints.append(flow == 1)

In [ ]:
prob = gcs.solve_from_ilp(ilp_constraints)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

In [ ]:
def plot_tour(gcs):
    for vertex in gcs.vertices:
        if np.isclose(vertex.y.value, 1):
            x1, x2 = vertex.variables
            values = np.array([x1.value, x2.value]).T
            plt.plot(*values, c="b", linestyle="--")
    
plt.figure()
plot_floor(rooms, doors)
plot_tour(gcs)
plt.savefig("inspection.pdf", bbox_inches="tight")